In [3]:
# ----------------------------
# Step 0: imports & settings
# ----------------------------
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency, pointbiserialr

# plotting style
# ...existing code...
sns.set(style="whitegrid", context="notebook", font_scale=1.05)
plt.rcParams['figure.figsize'] = (10, 6)
# ...existing code...

In [5]:
# Step 1: load & quick read
# ----------------------------
DATA_PATH = "/Users/tanuridinethra/Downloads/Walmart_customer_final_10000.csv"   # <- change this to your file
df = pd.read_csv(DATA_PATH)

print("\n--- Shape & dtypes ---")
print(df.shape)
print(df.dtypes)

print("\n--- First 5 rows ---")
print(df.head())



--- Shape & dtypes ---
(10000, 24)
Product_ID               object
Product_Name             object
Brand                    object
Model                    object
Category                 object
Market_Price            float64
Purchase_Amount         float64
Discount_Applied         object
Rating                  float64
Feedback                 object
Customer_ID              object
Age                       int64
Gender                   object
City                     object
Purchase_Date            object
Payment_Method           object
Repeat_Customer          object
Competitor_Name          object
Competitor_Model         object
Competitor_Price          int64
Competitor_Rating       float64
Promotion_Competitor     object
Market_Share              int64
Competitor_Feedback      object
dtype: object

--- First 5 rows ---
  Product_ID Product_Name      Brand                          Model  \
0   PRD33554        Dress       Nike         Nike Tech Fleece Dress   
1   PRD12448   Sma

In [6]:
# Step 2: normalize column names & basic cleaning
# ----------------------------
# If column names have stray spaces or inconsistent case, normalize them:
df.columns = [c.strip() for c in df.columns]

# Remove leading/trailing spaces in string columns (safe operation)
for c in df.select_dtypes(include=["object"]).columns:
    df[c] = df[c].astype(str).str.strip()

# Standardize obvious binary values (e.g., Repeat_Customer, Discount_Applied)
# If your dataset uses Yes/No variants, lowercase them to unify
if 'Repeat_Customer' in df.columns:
    df['Repeat_Customer'] = df['Repeat_Customer'].str.lower().replace({'yes':'yes','y':'yes','no':'no','n':'no'})

if 'Discount_Applied' in df.columns:
    df['Discount_Applied'] = df['Discount_Applied'].str.lower().replace({'yes':'yes','y':'yes','no':'no','n':'no'})


In [7]:
# Step 3: convert data types
# ----------------------------
# Dates
if 'Purchase_Date' in df.columns:
    df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'], errors='coerce')

# Numeric conversions (force numeric, coerce errors -> NaN)
num_cols = ['Market_Price', 'Purchase_Amount', 'Age', 'Competitor_Price',
            'Competitor_Rating', 'Market_Share', 'Rating']
num_cols = [c for c in num_cols if c in df.columns]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')


In [9]:
# Step 4: missing values analysis
# ----------------------------
print("\n--- Missing values (count & percent) ---")
missing = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_pct': df.isna().mean() * 100
}).sort_values('missing_pct', ascending=False)
print(missing[missing['missing_count'] > 0])

# Visual missingness matrix (saves an image)

plt.title("Missing values matrix")
plt.tight_layout()
plt.savefig("missing_matrix.png")
plt.close()


--- Missing values (count & percent) ---
Empty DataFrame
Columns: [missing_count, missing_pct]
Index: []


In [10]:
# Step 5: numeric feature overview
# ----------------------------
print("\n--- Numeric descriptive stats ---")
print(df[num_cols].describe().T)

# Histograms + boxplots for each numeric feature
for col in num_cols:
    fig, axes = plt.subplots(1, 2, figsize=(12,4))
    sns.histplot(df[col].dropna(), kde=True, ax=axes[0])
    axes[0].set_title(f"Distribution — {col}")
    sns.boxplot(x=df[col], ax=axes[1])
    axes[1].set_title(f"Boxplot — {col}")
    plt.tight_layout()
    plt.savefig(f"numeric_{col}.png")
    plt.close()


--- Numeric descriptive stats ---
                     count        mean         std    min     25%      50%  \
Market_Price       10000.0  294.191455  164.624813  11.21  147.72  295.270   
Purchase_Amount    10000.0  268.488960  154.323268   8.88  135.13  262.945   
Age                10000.0   38.758200   12.360656  18.00   28.00   39.000   
Competitor_Price   10000.0  108.682000  119.723165  50.00   50.00   60.000   
Competitor_Rating  10000.0    3.850360    0.408811   2.90    3.60    3.900   
Market_Share       10000.0   24.181600   10.357468  10.00   20.00   20.000   
Rating             10000.0    3.980690    0.407898   2.90    3.70    4.000   

                      75%     max  
Market_Price       434.33  620.11  
Purchase_Amount    390.50  616.77  
Age                 49.00   60.00  
Competitor_Price    70.00  520.00  
Competitor_Rating    4.20    4.70  
Market_Share        30.00   45.00  
Rating               4.30    4.70  


In [11]:
# Step 6: categorical feature overview
# ----------------------------
cat_cols = [c for c in df.columns if df[c].dtype == "object" and c not in ['Repeat_Customer']]
print("\nCategorical columns:", cat_cols)

# Frequency tables and top categories (top 10)
for c in cat_cols:
    print(f"\n--- Top values for {c} ---")
    print(df[c].value_counts(dropna=False).head(10))

# Plot top brands and categories if present
if 'Brand' in df.columns:
    top_brands = df['Brand'].value_counts().nlargest(15)
    sns.barplot(y=top_brands.index, x=top_brands.values)
    plt.title("Top 15 Brands by count")
    plt.xlabel("Count")
    plt.ylabel("Brand")
    plt.tight_layout()
    plt.savefig("top_brands.png")
    plt.close()

if 'Category' in df.columns:
    top_cat = df['Category'].value_counts().nlargest(15)
    sns.barplot(y=top_cat.index, x=top_cat.values)
    plt.title("Top 15 Categories by count")
    plt.xlabel("Count")
    plt.ylabel("Category")
    plt.tight_layout()
    plt.savefig("top_categories.png")
    plt.close()



Categorical columns: ['Product_ID', 'Product_Name', 'Brand', 'Model', 'Category', 'Discount_Applied', 'Feedback', 'Customer_ID', 'Gender', 'City', 'Payment_Method', 'Competitor_Name', 'Competitor_Model', 'Promotion_Competitor', 'Competitor_Feedback']

--- Top values for Product_ID ---
Product_ID
PRD44193    8
PRD41817    8
PRD21309    7
PRD33580    6
PRD03503    6
PRD45238    6
PRD38416    6
PRD45261    6
PRD13253    6
PRD05228    6
Name: count, dtype: int64

--- Top values for Product_Name ---
Product_Name
Smartphone    687
Face Cream    668
Headphones    665
Laptop        664
Dress         646
T-Shirt       642
Shampoo       628
Lamp          628
Sofa Cover    623
Jeans         623
Name: count, dtype: int64

--- Top values for Brand ---
Brand
Levi's       675
Apple        667
LG           666
Puma         659
Whirlpool    652
Dove         649
L'Oreal      634
Ikea         632
Sony         623
Samsung      614
Name: count, dtype: int64

--- Top values for Model ---
Model
Whirlpool LE

In [13]:
#Step 7: target variable check (Repeat_Customer)
# ----------------------------
if 'Repeat_Customer' in df.columns:
    print("\n--- Repeat_Customer distribution ---")
    print(df['Repeat_Customer'].value_counts(dropna=False))
    print(df['Repeat_Customer'].value_counts(normalize=True, dropna=True) * 100)

    # Visualize
    sns.countplot(data=df, x='Repeat_Customer', order=df['Repeat_Customer'].value_counts().index)
    plt.title("Repeat_Customer distribution")
    plt.tight_layout()
    plt.savefig("repeat_customer_dist.png")
    plt.close()

    # Create binary column for numeric correlation calculations
    df['Repeat_Customer_bin'] = df['Repeat_Customer'].map({'yes': 1, 'no': 0})



--- Repeat_Customer distribution ---
Repeat_Customer
no     5129
yes    4871
Name: count, dtype: int64
Repeat_Customer
no     51.29
yes    48.71
Name: proportion, dtype: float64


In [14]:
# Step 8: correlation analysis (numeric & target)
# ----------------------------
# Numeric correlation matrix
if len(num_cols) >= 2:
    corr = df[num_cols].corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='vlag', center=0)
    plt.title("Pearson correlation (numeric)")
    plt.tight_layout()
    plt.savefig("numeric_corr.png")
    plt.close()
    print("\n--- Numeric correlations ---")
    print(corr)

# Point-biserial correlation between binary Repeat_Customer and numeric features
if 'Repeat_Customer_bin' in df.columns:
    print("\n--- Point-biserial correlations with Repeat_Customer ---")
    for col in num_cols:
        mask = df['Repeat_Customer_bin'].notna() & df[col].notna()
        if mask.sum() > 10:
            r, p = pointbiserialr(df.loc[mask, 'Repeat_Customer_bin'], df.loc[mask, col])
            print(f"{col}: r={r:.3f}, p={p:.3g}")
        else:
            print(f"{col}: not enough paired data")



--- Numeric correlations ---
                   Market_Price  Purchase_Amount       Age  Competitor_Price  \
Market_Price           1.000000         0.975957  0.007575          0.002409   
Purchase_Amount        0.975957         1.000000  0.009188          0.001400   
Age                    0.007575         0.009188  1.000000          0.010604   
Competitor_Price       0.002409         0.001400  0.010604          1.000000   
Competitor_Rating      0.009911         0.009298  0.023116          0.109096   
Market_Share          -0.009588        -0.008969  0.020496          0.300571   
Rating                 0.004709         0.006074 -0.001862          0.264644   

                   Competitor_Rating  Market_Share    Rating  
Market_Price                0.009911     -0.009588  0.004709  
Purchase_Amount             0.009298     -0.008969  0.006074  
Age                         0.023116      0.020496 -0.001862  
Competitor_Price            0.109096      0.300571  0.264644  
Competitor_Rat

In [15]:
# Step 9: categorical vs Repeat_Customer (Cramér's V)
# ----------------------------
def cramers_v(x, y):
    """Compute Cramér's V statistic for categorical-categorical association."""
    confusion = pd.crosstab(x, y)
    if confusion.size == 0:
        return np.nan
    chi2 = chi2_contingency(confusion, correction=False)[0]
    n = confusion.sum().sum()
    if n == 0:
        return np.nan
    phi2 = chi2 / n
    r, k = confusion.shape
    # bias correction
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    denom = min((kcorr-1), (rcorr-1))
    if denom == 0:
        return np.nan
    return np.sqrt(phi2corr / denom)

if 'Repeat_Customer' in df.columns:
    print("\n--- Cramér's V: categorical vs Repeat_Customer ---")
    cat_cols_for_cv = [c for c in cat_cols if df[c].nunique() < 200]  # limit very high-card columns
    cramers_results = {}
    for c in cat_cols_for_cv:
        try:
            v = cramers_v(df[c], df['Repeat_Customer'])
            cramers_results[c] = v
        except Exception as e:
            cramers_results[c] = np.nan

    cramers_df = pd.Series(cramers_results).sort_values(ascending=False)
    print(cramers_df.head(20))


--- Cramér's V: categorical vs Repeat_Customer ---
Model                   0.180022
Competitor_Model        0.155645
Category                0.151387
Competitor_Name         0.094887
Competitor_Feedback     0.052859
Product_Name            0.040993
Feedback                0.030014
Brand                   0.024449
Discount_Applied        0.016692
Payment_Method          0.012810
Gender                  0.000000
Promotion_Competitor    0.000000
dtype: float64


In [16]:
# Step 10: outlier detection (IQR method)
# ----------------------------
outlier_summary = []
for c in num_cols:
    col_series = df[c].dropna()
    if col_series.empty:
        continue
    Q1 = col_series.quantile(0.25)
    Q3 = col_series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    n_out = ((df[c] < lower) | (df[c] > upper)).sum()
    outlier_summary.append((c, n_out, lower, upper))

print("\n--- Outlier summary (IQR method) ---")
for c, n_out, lower, upper in outlier_summary:
    print(f"{c}: outliers={n_out}, lower={lower:.2f}, upper={upper:.2f}")


--- Outlier summary (IQR method) ---
Market_Price: outliers=0, lower=-282.20, upper=864.25
Purchase_Amount: outliers=0, lower=-247.93, upper=773.56
Age: outliers=0, lower=-3.50, upper=80.50
Competitor_Price: outliers=1871, lower=20.00, upper=100.00
Competitor_Rating: outliers=0, lower=2.70, upper=5.10
Market_Share: outliers=0, lower=5.00, upper=45.00
Rating: outliers=0, lower=2.80, upper=5.20


In [17]:
# Step 11: business-focused cross-tabs & the specific repeat-customer check
# ----------------------------
# 1) For each repeat customer, how many times they bought same Brand+Category
if 'Repeat_Customer_bin' in df.columns:
    repeat_df = df[df['Repeat_Customer_bin'] == 1].copy()
    # count purchases by customer, brand, category
    rep_group = repeat_df.groupby(['Customer_ID', 'Brand', 'Category']).size().reset_index(name='purchase_count')
    # customers who bought same Brand+Category more than once
    loyal_purchases = rep_group[rep_group['purchase_count'] > 1].copy()
    print("\n--- Repeat-customer loyalty (same Brand & Category >1 times) ---")
    print(loyal_purchases.head(20))
    # summary: how many unique customers per brand-category
    loyalty_summary = loyal_purchases.groupby(['Brand', 'Category'])['Customer_ID'].nunique().reset_index(name='num_unique_customers')
    print("\n--- Loyalty summary by Brand+Category ---")
    print(loyalty_summary.sort_values('num_unique_customers', ascending=False).head(20))
    # Save to CSV
    loyal_purchases.to_csv("repeat_same_brand_category_details.csv", index=False)
    loyalty_summary.to_csv("repeat_same_brand_category_summary.csv", index=False)

# 2) Compare purchase amounts for repeat vs non-repeat
if 'Repeat_Customer_bin' in df.columns and 'Purchase_Amount' in df.columns:
    summary_by_repeat = df.groupby('Repeat_Customer_bin')['Purchase_Amount'].agg(['count', 'mean', 'median', 'std']).reset_index()
    print("\n--- Purchase_Amount summary by Repeat_Customer ---")
    print(summary_by_repeat)

# ----------------------------
# Step 12: recommended next steps & feature engineering hints
# ----------------------------
print("""
--- Recommended next steps (short checklist) ---
1) If your target is Repeat_Customer -> it's a classification problem:
   - Handle class imbalance (SMOTE, class_weight, undersampling)
   - Categorical encoding: target/mean encoding for high-cardinality (Brand, Model),
     One-Hot for low-cardinality (Payment_Method, Discount_Applied)
   - Use point-biserial/cramers results to prioritize features.

2) If your target is Purchase_Amount -> regression:
   - Log-transform highly skewed amounts before modelling
   - Consider grouping Brand+Category into aggregated features (avg_price_by_brand, brand_market_share)

3) Save cleaned dataset for modelling:
   df.to_csv("cleaned_dataset_for_modeling.csv", index=False)

4) If you won't train locally and will use an API:
   - Export a 'model-ready' CSV with proper encodings or with raw fields that the API supports.
   - Provide a data dictionary alongside the CSV.

Files saved by this script:
 - missing_matrix.png
 - numeric_*.png (per numeric column)
 - top_brands.png / top_categories.png
 - repeat_same_brand_category_details.csv
 - repeat_same_brand_category_summary.csv
""")

# Save cleaned dataframe for downstream modelling
os.makedirs("analysis_outputs", exist_ok=True)
df.to_csv(os.path.join("analysis_outputs", "cleaned_dataset_for_modeling.csv"), index=False)



--- Repeat-customer loyalty (same Brand & Category >1 times) ---
                             Customer_ID      Brand     Category  \
0   0020b0f3-7120-45f6-b5c3-e037ae90d459       Nike  Electronics   
2   0040b404-b18b-4fa6-8e94-5804f29ade39     Levi's       Beauty   
3   005a074d-c082-445d-9307-c90a37daccaa       Puma  Electronics   
4   0083a488-4956-4890-98f2-8e957a69c9d6    L'Oreal  Electronics   
6   008992e4-eeeb-4515-8c8d-c59c93b5e54c         LG  Electronics   
8   00dc7112-0b13-4442-a2d0-13238e9f8da4       Ikea       Beauty   
10  0107bae7-246e-4908-94b3-a6d1dbf3d1fa      Apple  Electronics   
12  01364432-f904-4901-bdfd-1df67d4de4b1       Dove     Clothing   
13  014a0bd5-4b43-4b99-bf45-77f6041eda34         LG  Electronics   
14  01a8ce6d-2a59-45cf-8a3f-2f852553dd3d       Nike     Clothing   
16  01cd83fd-17c2-4743-895f-285c2c5e6e88       Ikea  Electronics   
17  01dae9fb-dce0-4e26-b528-48904af85117  Panasonic       Beauty   
18  01e217bc-2ec3-495f-97f0-9391fabf51b6    Samsun

In [18]:
#discount and prediction analysis

import os
import pandas as pd
import numpy as np

DATA_PATH = "/Users/tanuridinethra/Downloads/Walmart_customer_final_10000.csv"  # <-- change to your CSV

os.makedirs("analysis_outputs", exist_ok=True)


In [19]:
# ----------------------------
# Load & basic cleaning
# ----------------------------
df = pd.read_csv(DATA_PATH)

# strip spaces, normalize key fields
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip()

# types
if "Purchase_Date" in df.columns:
    df["Purchase_Date"] = pd.to_datetime(df["Purchase_Date"], errors="coerce")

num_cols = ["Market_Price", "Purchase_Amount", "Age", "Competitor_Price",
            "Competitor_Rating", "Market_Share", "Rating"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# normalize booleans
for c in ["Repeat_Customer", "Discount_Applied", "Promotion_Competitor"]:
    if c in df.columns:
        df[c] = df[c].str.lower().replace({"y":"yes","n":"no"})

In [20]:
# ----------------------------
# Filter to last 4 years (relative to max date in data)
# ----------------------------
max_date = df["Purchase_Date"].max()
if pd.isna(max_date):
    raise ValueError("Purchase_Date is missing/invalid; please fix dates.")

start_date = max_date - pd.DateOffset(years=4)
df4 = df[df["Purchase_Date"] >= start_date].copy()

In [21]:
# ----------------------------
# Analysis: among repeat customers, who bought same Brand+Category multiple times?
# ----------------------------
rep = df4[df4["Repeat_Customer"].str.lower() == "yes"].copy()

# count by (Customer_ID, Brand, Category)
repeat_counts = (
    rep.groupby(["Customer_ID", "Brand", "Category"])
      .size().reset_index(name="purchase_count")
)

loyal_multi = repeat_counts[repeat_counts["purchase_count"] > 1].copy()

summary_brand_cat = (
    loyal_multi.groupby(["Brand", "Category"])["Customer_ID"]
    .nunique().reset_index(name="num_unique_repeat_customers")
    .sort_values("num_unique_repeat_customers", ascending=False)
)

summary_brand_cat.to_csv("analysis_outputs/repeat_same_brand_category_summary.csv", index=False)

print("Saved: analysis_outputs/repeat_same_brand_category_summary.csv")
print(summary_brand_cat.head(15))


Saved: analysis_outputs/repeat_same_brand_category_summary.csv
        Brand     Category  num_unique_repeat_customers
5       Apple  Electronics                           67
17         LG  Electronics                           60
47  Whirlpool  Electronics                           57
38    Samsung  Electronics                           55
20     Levi's  Electronics                           55
11       Ikea  Electronics                           54
39    Sephora       Beauty                           53
44       Sony  Electronics                           50
2      Adidas  Electronics                           49
29  Panasonic  Electronics                           49
35       Puma  Electronics                           47
32    Philips  Electronics                           47
12    L'Oreal       Beauty                           47
6        Dove       Beauty                           40
24      Nivea       Beauty                           39


In [22]:
# ----------------------------
# Build sequence labels for prediction
# Label = 1 if NEXT purchase (same Category) keeps the SAME Brand, else 0 (switch)
# Only consider customers who are repeat (historically) and have >=2 purchases in that Category
# ----------------------------
use_cols = [
    "Customer_ID","Purchase_Date","Brand","Category","City","Gender","Age",
    "Discount_Applied","Market_Price","Purchase_Amount","Competitor_Price",
    "Competitor_Rating","Payment_Method","Market_Share","Rating","Repeat_Customer"
]
use_cols = [c for c in use_cols if c in df4.columns]
data = df4[use_cols].dropna(subset=["Customer_ID","Category","Brand","Purchase_Date"]).copy()

# sort and create next-row info within (Customer, Category)
data = data.sort_values(["Customer_ID","Category","Purchase_Date"])
data["next_brand"] = (
    data.groupby(["Customer_ID","Category"])["Brand"]
        .shift(-1)
)
data["next_date"] = (
    data.groupby(["Customer_ID","Category"])["Purchase_Date"]
        .shift(-1)
)

# keep rows where a "next" exists
has_next = data[~data["next_brand"].isna()].copy()
has_next["label_loyal"] = (has_next["Brand"] == has_next["next_brand"]).astype(int)

# ------------- Feature engineering (at time t, before we see t+1)
# per-customer frequency signals up to current time
def add_customer_signals(g):
    g = g.sort_values("Purchase_Date")
    g["cust_total_so_far"]  = np.arange(1, len(g)+1)  # 1..n
    return g

data = data.groupby("Customer_ID", group_keys=False).apply(add_customer_signals)

# per-customer-category frequency signals
def add_cat_signals(g):
    g = g.sort_values("Purchase_Date")
    g["cust_cat_total_so_far"] = np.arange(1, len(g)+1)
    return g

data = data.groupby(["Customer_ID","Category"], group_keys=False).apply(add_cat_signals)

# per-customer-brand-category frequency & share
def add_brand_cat_signals(g):
    g = g.sort_values("Purchase_Date")
    g["cust_brand_cat_total_so_far"] = np.arange(1, len(g)+1)
    return g

data = data.groupby(["Customer_ID","Category","Brand"], group_keys=False).apply(add_brand_cat_signals)

# join the engineered features onto has_next (aligning by current row)
feat_cols = [
    "cust_total_so_far","cust_cat_total_so_far","cust_brand_cat_total_so_far"
]
has_next = has_next.merge(
    data[["Customer_ID","Category","Brand","Purchase_Date"] + feat_cols],
    on=["Customer_ID","Category","Brand","Purchase_Date"],
    how="left"
)

# recency: days until next purchase (can be used for survival-like targets later; keep for reference)
has_next["days_to_next"] = (has_next["next_date"] - has_next["Purchase_Date"]).dt.days

# price/competitor gaps (optional numeric signals)
if "Competitor_Price" in has_next.columns:
    has_next["gap_vs_competitor"] = has_next["Purchase_Amount"] - has_next["Competitor_Price"]
if "Market_Price" in has_next.columns:
    has_next["discount_amount"] = has_next["Market_Price"] - has_next["Purchase_Amount"]

# we’ll keep a compact training set:
keep_model_cols = [
    # label
    "label_loyal",
    # IDs/time for tracing (not used for training by AutoML unless included)
    "Customer_ID","Purchase_Date","next_date",
    # categorical/context
    "Brand","Category","City","Gender","Payment_Method","Discount_Applied","Repeat_Customer",
    # numeric
    "Age","Market_Price","Purchase_Amount","Competitor_Price","Competitor_Rating",
    "Market_Share","Rating","cust_total_so_far","cust_cat_total_so_far","cust_brand_cat_total_so_far",
    "gap_vs_competitor","discount_amount","days_to_next"
]
keep_model_cols = [c for c in keep_model_cols if c in has_next.columns]

train_df = has_next[keep_model_cols].dropna(subset=["label_loyal"]).copy()

# Save training table
train_path = "analysis_outputs/loyalty_training.csv"
train_df.to_csv(train_path, index=False)
print(f"Saved training table: {train_path}  (rows: {len(train_df)})")

/var/folders/pk/xd0rqtps1qlfh1s45lmks7t40000gn/T/ipykernel_85401/2690488820.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby("Customer_ID", group_keys=False).apply(add_customer_signals)
/var/folders/pk/xd0rqtps1qlfh1s45lmks7t40000gn/T/ipykernel_85401/2690488820.py:44: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby(["Customer_ID","Category"], group_keys=False).apply(add_cat_s

Saved training table: analysis_outputs/loyalty_training.csv  (rows: 15334)


/var/folders/pk/xd0rqtps1qlfh1s45lmks7t40000gn/T/ipykernel_85401/2690488820.py:52: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby(["Customer_ID","Category","Brand"], group_keys=False).apply(add_brand_cat_signals)


In [23]:
# ----------------------------
# Inference candidates = last known purchase per (Customer, Category)
# Use these rows to predict “will next be same brand?”
# ----------------------------
last_rows = (
    data.sort_values("Purchase_Date")
        .groupby(["Customer_ID","Category"], as_index=False)
        .tail(1)
)

# align with same features
cand = last_rows.copy()
for col in ["gap_vs_competitor","discount_amount"]:
    if col not in cand.columns:
        cand[col] = np.nan
if "Competitor_Price" in cand.columns and "Purchase_Amount" in cand.columns:
    cand["gap_vs_competitor"] = cand["Purchase_Amount"] - cand["Competitor_Price"]
if "Market_Price" in cand.columns and "Purchase_Amount" in cand.columns:
    cand["discount_amount"] = cand["Market_Price"] - cand["Purchase_Amount"]

cand = cand[[c for c in keep_model_cols if c != "label_loyal" and c != "next_date" and c != "days_to_next"]]
cand_path = "analysis_outputs/inference_candidates.csv"
cand.to_csv(cand_path, index=False)
print(f"Saved inference candidates: {cand_path}  (rows: {len(cand)})")

Saved inference candidates: analysis_outputs/inference_candidates.csv  (rows: 5000)


In [24]:
#Repeat Customer & Discount Analysis

import pandas as pd

# Load your dataset
df = pd.read_csv("/Users/tanuridinethra/Downloads/Walmart_customer_final_10000.csv")

# Filter repeat customers
repeat_df = df[df['Repeat_Customer'] == 'Yes']


In [25]:
# Count how many times a repeat customer bought the same brand/category
repeat_analysis = (
    repeat_df.groupby(['Customer_ID', 'Brand', 'Category'])
    .size()
    .reset_index(name='Purchase_Count')
    .sort_values(['Customer_ID','Purchase_Count'], ascending=[True, False])
)

print(repeat_analysis.head(10))


                            Customer_ID      Brand     Category  \
0  0020b0f3-7120-45f6-b5c3-e037ae90d459       Nike  Electronics   
1  004011cf-4b63-4455-bd67-6ade8beb6c0a       Dove       Beauty   
2  0040b404-b18b-4fa6-8e94-5804f29ade39     Levi's       Beauty   
3  005a074d-c082-445d-9307-c90a37daccaa       Puma  Electronics   
4  0083a488-4956-4890-98f2-8e957a69c9d6    L'Oreal  Electronics   
5  008772bd-4e1b-458a-b20e-b0a5591999d6    Philips  Electronics   
6  008992e4-eeeb-4515-8c8d-c59c93b5e54c         LG  Electronics   
7  00bf711e-a00e-4c07-852a-0cf8520d5644  Whirlpool       Beauty   
8  00dc7112-0b13-4442-a2d0-13238e9f8da4       Ikea       Beauty   
9  0106c31a-e5ff-4b57-973a-54e46affc3c4       Sony  Electronics   

   Purchase_Count  
0               2  
1               1  
2               2  
3               2  
4               2  
5               1  
6               2  
7               1  
8               2  
9               1  


In [26]:
# Count how many repeat purchases were made with/without discount
discount_analysis = (
    repeat_df.groupby(['Customer_ID', 'Brand', 'Category', 'Discount_Applied'])
    .size()
    .reset_index(name='Purchase_Count')
)

print(discount_analysis.head(10))


                            Customer_ID      Brand     Category  \
0  0020b0f3-7120-45f6-b5c3-e037ae90d459       Nike  Electronics   
1  004011cf-4b63-4455-bd67-6ade8beb6c0a       Dove       Beauty   
2  0040b404-b18b-4fa6-8e94-5804f29ade39     Levi's       Beauty   
3  005a074d-c082-445d-9307-c90a37daccaa       Puma  Electronics   
4  0083a488-4956-4890-98f2-8e957a69c9d6    L'Oreal  Electronics   
5  008772bd-4e1b-458a-b20e-b0a5591999d6    Philips  Electronics   
6  008992e4-eeeb-4515-8c8d-c59c93b5e54c         LG  Electronics   
7  00bf711e-a00e-4c07-852a-0cf8520d5644  Whirlpool       Beauty   
8  00dc7112-0b13-4442-a2d0-13238e9f8da4       Ikea       Beauty   
9  0106c31a-e5ff-4b57-973a-54e46affc3c4       Sony  Electronics   

  Discount_Applied  Purchase_Count  
0               No               2  
1              Yes               1  
2              Yes               2  
3              Yes               2  
4               No               2  
5               No               1  
6  

In [27]:
# Example: summarize per customer-brand-category
api_input = discount_analysis.groupby(['Customer_ID', 'Brand', 'Category']).agg({
    'Purchase_Count':'sum',
    'Discount_Applied': lambda x: list(x)  # list of Yes/No
}).reset_index()

print(api_input.head())


                            Customer_ID    Brand     Category  Purchase_Count  \
0  0020b0f3-7120-45f6-b5c3-e037ae90d459     Nike  Electronics               2   
1  004011cf-4b63-4455-bd67-6ade8beb6c0a     Dove       Beauty               1   
2  0040b404-b18b-4fa6-8e94-5804f29ade39   Levi's       Beauty               2   
3  005a074d-c082-445d-9307-c90a37daccaa     Puma  Electronics               2   
4  0083a488-4956-4890-98f2-8e957a69c9d6  L'Oreal  Electronics               2   

  Discount_Applied  
0             [No]  
1            [Yes]  
2            [Yes]  
3            [Yes]  
4             [No]  


In [30]:
def predict_repeat(customer_id, brand, category, purchase_count, discount_history):
    """
    Simulate repeat purchase prediction:
    - If the customer ever got a discount before, predict True
    - If the customer purchased more than 2 times, predict True
    - Else predict False
    """
    if "Yes" in discount_history:
        return True
    elif purchase_count > 2:
        return True
    else:
        return False

# Step 5b: Apply dummy prediction
predictions = []
for _, row in api_input.iterrows():
    predicted_repeat = predict_repeat(
        row['Customer_ID'],
        row['Brand'],
        row['Category'],
        row['Purchase_Count'],
        row['Discount_Applied']
    )
    predictions.append({
        "Customer_ID": row['Customer_ID'],
        "Brand": row['Brand'],
        "Category": row['Category'],
        "Predicted_Repeat": predicted_repeat
    })


In [31]:
# Step 6: Create final DataFrame with predictions
pred_df = pd.DataFrame(predictions)
print("\nPredicted repeat purchase results:")
print(pred_df.head(10))


Predicted repeat purchase results:
                            Customer_ID      Brand     Category  \
0  0020b0f3-7120-45f6-b5c3-e037ae90d459       Nike  Electronics   
1  004011cf-4b63-4455-bd67-6ade8beb6c0a       Dove       Beauty   
2  0040b404-b18b-4fa6-8e94-5804f29ade39     Levi's       Beauty   
3  005a074d-c082-445d-9307-c90a37daccaa       Puma  Electronics   
4  0083a488-4956-4890-98f2-8e957a69c9d6    L'Oreal  Electronics   
5  008772bd-4e1b-458a-b20e-b0a5591999d6    Philips  Electronics   
6  008992e4-eeeb-4515-8c8d-c59c93b5e54c         LG  Electronics   
7  00bf711e-a00e-4c07-852a-0cf8520d5644  Whirlpool       Beauty   
8  00dc7112-0b13-4442-a2d0-13238e9f8da4       Ikea       Beauty   
9  0106c31a-e5ff-4b57-973a-54e46affc3c4       Sony  Electronics   

   Predicted_Repeat  
0             False  
1              True  
2              True  
3              True  
4             False  
5             False  
6             False  
7             False  
8             False  
9     